# Bigcontest 데이터 탐색

## 0. library Packages

In [1]:
import pandas as pd
import numpy as np

## 1. Import Data

In [2]:
loan_result = pd.read_csv("../../../../bigcontest/data/loan_result.csv")
log_data = pd.read_csv("../../../../bigcontest/data/log_data.csv")
user_spec = pd.read_csv("../../../../bigcontest/data/user_spec_new.csv")

## 2. Preview & Preprocessing

### 2-1) loan_result

![loan_result](../img/loan_result_describe.png)

In [3]:
loan_result

,application_id,loanapply_insert_time,bank_id,product_id,loan_limit,loan_rate,is_applied
0,1748340,2022-06-07 13:05:41,7,191,42000000.0,13.6,NaN
1,1748340,2022-06-07 13:05:41,25,169,24000000.0,17.9,NaN
2,1748340,2022-06-07 13:05:41,2,7,24000000.0,18.5,NaN
3,1748340,2022-06-07 13:05:41,4,268,29000000.0,10.8,NaN
4,1748340,2022-06-07 13:05:41,11,118,5000000.0,16.4,NaN
...,...,...,...,...,...,...,...
13527358,1428218,2022-06-03 12:01:58,62,200,3000000.0,14.8,NaN
13527359,1428218,2022-06-03 12:01:49,2,7,40000000.0,11.8,NaN
13527360,1428218,2022-06-03 12:01:48,32,257,15000000.0,7.2,NaN
13527361,1428218,2022-06-03 12:01:48,33,110,44000000.0,13.5,NaN


13,527,363 행과 7열로 데이터의 행 개수가 많음

In [4]:
loan_result.isnull().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                  7495
loan_rate                   7495
is_applied               3257239
dtype: int64

In [7]:
appid_list = user_spec['application_id'].unique()
loan_result = loan_result.astype({"loanapply_insert_time":'datetime64'}).loc[(loan_result['loan_limit'].notnull()) & (loan_result['application_id'].isin(appid_list)),:]

In [9]:
loan_result.isnull().sum()

application_id                 0
loanapply_insert_time          0
bank_id                        0
product_id                     0
loan_limit                     0
loan_rate                      0
is_applied               3224661
dtype: int64

한도조회 일시를 나타내는 `loanapply_insert_time` 변수가 object 형태로 저장되어 있어 `datetime64`로 정리함.  
`loan_limit`과 `loan_rate`가 결측인 경우를 제외한다. 그리고, `user_spec`에 없는 `application_id`를 가진 경우 또한 제외한다.

In [5]:
loan_result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13527363 entries, 0 to 13527362
Data columns (total 7 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   application_id         int64         
 1   loanapply_insert_time  datetime64[ns]
 2   bank_id                int64         
 3   product_id             int64         
 4   loan_limit             float64       
 5   loan_rate              float64       
 6   is_applied             float64       
dtypes: datetime64[ns](1), float64(3), int64(3)
memory usage: 722.4 MB


In [10]:
loan_result_train = loan_result.loc[loan_result["is_applied"].notnull(),]
loan_result_test = loan_result.loc[loan_result["is_applied"].isnull(),]

In [11]:
print(loan_result["is_applied"].value_counts())
print(loan_result["is_applied"].value_counts(normalize=True))

0.0    9636353
1.0     551045
Name: is_applied, dtype: int64
0.0    0.945909
1.0    0.054091
Name: is_applied, dtype: float64


NaN 값이 아닌 **신청 여부(is_applied)** 값들 중 대략 94.5%가 신청을 하지 않았고, 5.5%가 신청을 하였다.  
1의 비율이 적으므로 이를 고려할 수 있는 모델을 만드는 것이 중요해보인다.

### 2-2) log_data

![log_data](../img/log_data_describe.png)

In [8]:
log_data

,user_id,event,timestamp,mp_os,mp_app_version,date_cd
0,576409,StartLoanApply,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
1,576409,ViewLoanApplyIntro,2022-03-25 11:12:09,Android,3.8.2,2022-03-25
2,72878,EndLoanApply,2022-03-25 11:14:44,Android,3.8.4,2022-03-25
3,645317,OpenApp,2022-03-25 11:15:09,iOS,3.6.1,2022-03-25
4,645317,UseLoanManage,2022-03-25 11:15:11,iOS,3.6.1,2022-03-25
...,...,...,...,...,...,...
17843988,242442,StartLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
17843989,242442,EndLoanApply,2022-05-27 16:08:21,Android,3.13.0,2022-05-27
17843990,593062,OpenApp,2022-05-27 16:19:10,Android,3.13.0,2022-05-27
17843991,593062,Login,2022-05-27 16:19:15,Android,3.13.0,2022-05-27


17,843,993 행과 6열로 데이터의 행 개수가 많음

In [9]:
log_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17843993 entries, 0 to 17843992
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   user_id         int64 
 1   event           object
 2   timestamp       object
 3   mp_os           object
 4   mp_app_version  object
 5   date_cd         object
dtypes: int64(1), object(5)
memory usage: 816.8+ MB


In [10]:
log_data = log_data.drop(labels = ["mp_os", "mp_app_version","date_cd"], axis = 1).astype({'timestamp':'datetime64'})

행동일시를 나타내는 `timestamp` 가 object 형태이므로 `datetime64`로 정리할 필요가 있어보임. 그리고, 사용 기종과 어플리케이션 버전이 크게 유의미하지 않을 것이라 생각되고, `data_cd`는 `timestamp`에 포함되므로 삭제한다.

In [11]:
log_data["event"].value_counts()

OpenApp                    3460762
EndLoanApply               2715253
GetCreditInfo              2661997
Login                      2463755
StartLoanApply             1893914
ViewLoanApplyIntro         1804712
UseLoanManage              1558906
CompleteIDCertification    1237777
SignUp                       34892
UsePrepayCalc                 7360
UseDSRCalc                    4665
Name: event, dtype: int64

In [17]:
log_data.isnull().sum()

user_id      0
event        0
timestamp    0
dtype: int64

### 2-3) user_spec

![user_spec](../img/user_spec_describe.png)

In [12]:
user_spec

,Unnamed: 0,application_id,user_id,insert_time,credit_score,yearly_income,income_type,company_enter_month,employment_type,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,birth_year,gender
0,1,1249046,118218,2022-06-07 06:28:18,660.0,108000000.0,PRIVATEBUSINESS,201511,기타,자가,1000000.0,기타,0.0,NaN,4,162000000.0,1985,1
1,2,954900,553686,2022-06-07 14:29:03,870.0,30000000.0,PRIVATEBUSINESS,200702,정규직,기타가족소유,30000000.0,대환대출,0.0,NaN,1,27000000.0,1968,1
2,3,137274,59516,2022-06-07 21:40:22,710.0,30000000.0,FREELANCER,202109,기타,기타가족소유,10000000.0,생활비,0.0,NaN,5,15000000.0,1997,1
3,4,1570936,167320,2022-06-07 09:40:27,820.0,62000000.0,EARNEDINCOME,201701,정규직,자가,2000000.0,생활비,0.0,NaN,7,344000000.0,1989,1
4,5,967833,33400,2022-06-07 08:55:07,630.0,36000000.0,EARNEDINCOME,202109,정규직,기타가족소유,5000000.0,생활비,0.0,0.0,1,16000000.0,2000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1384398,1384488,1864587,489900,2022-03-22 14:55:32,590.0,25000000.0,FREELANCER,202106,기타,기타가족소유,5000000.0,사업자금,NaN,NaN,0,NaN,2000,1
1384399,1384489,1327066,151422,2022-03-22 01:19:24,980.0,20000000.0,OTHERINCOME,0,기타,자가,50000000.0,생활비,NaN,NaN,1,NaN,1955,1
1384400,1384490,1319606,173524,2022-03-22 07:34:32,750.0,75000000.0,EARNEDINCOME,200908,정규직,자가,100000000.0,대환대출,NaN,NaN,8,200000000.0,1983,1
1384401,1384491,1482466,766546,2022-03-22 22:12:35,640.0,50000000.0,EARNEDINCOME,201705,정규직,자가,10000000.0,대환대출,NaN,NaN,10,117000000.0,1975,1


1,394,216 행과 17열로 각 행이 한 사람에 대한 정보를 뜻함

In [13]:
user_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384403 entries, 0 to 1384402
Data columns (total 18 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   Unnamed: 0                           1384403 non-null  int64  
 1   application_id                       1384403 non-null  int64  
 2   user_id                              1384403 non-null  int64  
 3   insert_time                          1384403 non-null  object 
 4   credit_score                         1280926 non-null  float64
 5   yearly_income                        1384403 non-null  float64
 6   income_type                          1384403 non-null  object 
 7   company_enter_month                  1384403 non-null  int64  
 8   employment_type                      1384403 non-null  object 
 9   houseown_type                        1384403 non-null  object 
 10  desired_amount                       1384403 non-null  float64
 11

In [14]:
user_spec = user_spec.astype({user_spec.columns[3]:'datetime64'}).drop('Unnamed: 0', axis = 1)

적절히 `dtype`을 변경하고, 시간을 나타내는 변수들에 대해서 적당히 `datetime64`로 변경한다.

In [15]:
user_spec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384403 entries, 0 to 1384402
Data columns (total 17 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   application_id                       1384403 non-null  int64         
 1   user_id                              1384403 non-null  int64         
 2   insert_time                          1384403 non-null  datetime64[ns]
 3   credit_score                         1280926 non-null  float64       
 4   yearly_income                        1384403 non-null  float64       
 5   income_type                          1384403 non-null  object        
 6   company_enter_month                  1384403 non-null  int64         
 7   employment_type                      1384403 non-null  object        
 8   houseown_type                        1384403 non-null  object        
 9   desired_amount                       1384403 non-null  fl

In [16]:
user_spec.isnull().sum()

application_id                               0
user_id                                      0
insert_time                                  0
credit_score                            103477
yearly_income                                0
income_type                                  0
company_enter_month                          0
employment_type                              0
houseown_type                                0
desired_amount                               0
purpose                                      0
personal_rehabilitation_yn              583505
personal_rehabilitation_complete_yn    1195356
existing_loan_cnt                            0
existing_loan_amt                       308292
birth_year                                   0
gender                                       0
dtype: int64

## `user_spec` vs `loan_result`

In [15]:
merged1 = pd.merge(user_spec, loan_result_train, how = 'inner', on = 'application_id')

In [16]:
merged1.columns

Index(['Unnamed: 0', 'application_id', 'user_id', 'insert_time',
       'credit_score', 'yearly_income', 'income_type', 'company_enter_month',
       'employment_type', 'houseown_type', 'desired_amount', 'purpose',
       'personal_rehabilitation_yn', 'personal_rehabilitation_complete_yn',
       'existing_loan_cnt', 'existing_loan_amt', 'birth_year', 'gender',
       'loanapply_insert_time', 'bank_id', 'product_id', 'loan_limit',
       'loan_rate', 'is_applied'],
      dtype='object')

In [18]:
colbar = {'정규직':1, '계약직':2, '일용직':3, '기타':4}

In [21]:
colbarr = merged1.replace({"employment_type":colbar})

In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('classic')

In [30]:
loan_result_train[loan_result_train['loan_limit'] == 0]['is_applied'].sum()

158.0

In [ ]:
scat = plt.scatter(np.log10(merged1['loan_limit']), merged1['loan_rate'], c = colbarr['employment_type'])
#plt.legend[scat ];

C:\Users\tspa9\anaconda3\lib\site-packages\pandas\core\arraylike.py:397: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
